In [ ]:
!pip install -q langchain_core langchain_openai langchain_community sentence-transformers langchain spacy 
!python -m spacy download en_core_web_sm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 7.4.0 requires pandocfilters>=1.4.1, which is not installed.

[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter, TokenTextSplitter
from langchain_community.vectorstores import FAISS
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from IPython.display import Latex
from langchain_community.llms import HuggingFacePipeline
from langchain_core.tools import tool
from langchain.agents import initialize_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import Tool

/home/vikram/Downloads/aban/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_default_device('cuda')

In [3]:
model_name="deepseek-ai/deepseek-math-7b-rl"
tokenizer=AutoTokenizer.from_pretrained(model_name)
deepseek_model=AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16,device_map="cuda")
deepseek_model.generation_config = GenerationConfig.from_pretrained(model_name)
deepseek_model.generation_config.pad_token_id=GenerationConfig.from_pretrained(model_name).eos_token_id

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


---

# Simple model run

In [12]:
messages = [
    {"role": "user", "content": "what is the integral of x^2 from 0 to 2?\nreason step by step, and put your final answer within \\boxed{}."}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text = tokenizer.decode(input_tensor[0], skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=500)
Latex(tokenizer.decode(outputs[0], skip_special_tokens=True))

<IPython.core.display.Latex object>

---

# Checking model's knowledge

In [13]:
content="""Answer the following question:
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads, and it is to be used in an application similar to the Joyce worm-gear screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(b) Find the torque required to raise and lower the load.\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text = tokenizer.decode(input_tensor[0], skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=5000)
Latex(tokenizer.decode(outputs[0], skip_special_tokens=True))

<IPython.core.display.Latex object>

In [14]:
content="""Answer the following question:
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads, and it is to be used in an application similar to the Joyce worm-gear screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text=tokenizer.decode(input_tensor[0],skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=5000)
Latex(tokenizer.decode(outputs[0], skip_special_tokens=True))

<IPython.core.display.Latex object>

---

# Complete context

In [15]:
context="""Finally, noting that the torque is the product of the force P and the mean radius dm/2,
for raising the load we can write
TR = Fdm
2
l + π f dm
πdm - f l
(8-1)
where TR is the torque required for two purposes: to overcome thread friction and to
raise the load.
The torque required to lower the load, from Eq. (f), is found to be
TL = Fdm
2
π f dm - l
πdm + f l 
(8-2)
This is the torque required to overcome a part of the friction in lowering the load. It may
turn out, in specific instances where the lead is large or the friction is low, that the load
will lower itself by causing the screw to spin without any external effort. In such cases, 
the torque TL from Eq. (8-2) will be negative or zero. When a positive torque is
obtained from this equation, the screw is said to be self-locking. Thus the condition
for self-locking is
π f dm > l
Now divide both sides of this inequality by πdm . Recognizing that l/πdm = tan λ, we
get
f > tan λ (8-3)
This relation states that self-locking is obtained whenever the coefficient of thread
friction is equal to or greater than the tangent of the thread lead angle.
An expression for efficiency is also useful in the evaluation of power screws. If
we let f = 0 in Eq. (8-1), we obtain
T0 = Fl
2π
(g)
which, since thread friction has been eliminated, is the torque required only to raise
the load. The efficiency is therefore
e = T0
TR
= Fl
2πTR
(8-4)
The preceding equations have been developed for square threads where the normal thread loads are parallel to the axis of the screw. In the case of Acme or other
threads, the normal thread load is inclined to the axis because of the thread angle 2α
and the lead angle λ. Since lead angles are small, this inclination can be neglected
and only the effect of the thread angle (Fig. 8-7a) considered. The effect of the angle
α is to increase the frictional force by the wedging action of the threads. Therefore
the frictional terms in Eq. (8-1) must be divided by cos α. For raising the load, or for
tightening a screw or bolt, this yields
TR = Fdm
2
l + π f dm sec α
πdm - f l sec α
(8-5)
In using Eq. (8-5), remember that it is an approximation because the effect of the
lead angle has been neglected.
For power screws, the Acme thread is not as efficient as the square thread, because
of the additional friction due to the wedging action, but it is often preferred because
it is easier to machine and permits the use of a split nut, which can be adjusted to
take up for wear.
Usually a third component of torque must be applied in power-screw applications.
When the screw is loaded axially, a thrust or collar bearing must be employed between
the rotating and stationary members in order to carry the axial component. Figure 8-7b
shows a typical thrust collar in which the load is assumed to be concentrated at the
mean collar diameter dc. If fc is the coefficient of collar friction, the torque required is
Tc = F fcdc/2 
(8-6)
"""

In [17]:
input_text = f"""Given the context:
<context>{context}<\context>\n
Answer the following question:
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads, and it is to be used in an application similar to the e Joyce worm-gear screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load.\n
Give the answer with step by step reasoning."""
input_ids = tokenizer(input_text, return_tensors="pt")
outputs = deepseek_model.generate(**input_ids,max_new_tokens=2000)
Latex(tokenizer.decode(outputs[0],skip_special_tokens=True))

<IPython.core.display.Latex object>

---

# RAG with raw text

In [5]:
loader = TextLoader("raw_text.txt", encoding="utf-8")
docs=loader.load()

In [20]:
splitter=RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

/tmp/ipykernel_1980553/3384329387.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [22]:
ques="""A square-thread power screw has a major diameter of 64 mm and a pitch of 4 mm with double threads, and it is to be used in screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 8 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load."""

similar=vectorstore.similarity_search(ques)
context=""
for txt in similar:
    context+=txt.page_content+"\n"

content=f"""Using the following context:<context>{context}<\context>\n
Answer the following question:<question>{ques}<\question>\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text=tokenizer.decode(input_tensor[0], skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=3000)

In [23]:
Latex(tokenizer.decode(outputs[0],skip_special_tokens=True))

<IPython.core.display.Latex object>

In [6]:
splitter=TokenTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

/tmp/ipykernel_2185142/3575699394.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [7]:
ques="""A square-thread power screw has a major diameter of 64 mm and a pitch of 4 mm with double threads, and it is to be used in screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 8 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load."""

similar=vectorstore.similarity_search(ques)
context=""
for txt in similar:
    context+=txt.page_content+"\n"

content=f"""Using the following context:<context>{context}<\context>\n
Answer the following question:<question>{ques}<\question>\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text=tokenizer.decode(input_tensor[0], skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=2000)

OutOfMemoryError: CUDA out of memory. Tried to allocate 68.00 MiB. GPU 0 has a total capacity of 15.72 GiB of which 65.31 MiB is free. Including non-PyTorch memory, this process has 15.26 GiB memory in use. Of the allocated memory 14.51 GiB is allocated by PyTorch, and 566.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

In [13]:
# !python -m spacy download en_core_web_sm

In [4]:
splitter=SpacyTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

/home/vikram/Downloads/aban/.venv/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Created a chunk of size 735, which is longer than the specified 700
Created a chunk of size 1337, which is longer than the specified 700
Created a chunk of size 716, which is longer than the specified 700
/tmp/ipykernel_1992606/1343699546.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=

In [7]:
ques="""A square-thread power screw has a major diameter of 64 mm and a pitch of 4 mm with double threads, and it is to be used in screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 8 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load."""

similar=vectorstore.similarity_search(ques)
context=""
for txt in similar:
    context+=txt.page_content+"\n"

content=f"""Using the following context:<context>{context}<\context>\n
Answer the following question:<question>{ques}<\question>\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text=tokenizer.decode(input_tensor[0],skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=3000)

In [8]:
Latex(tokenizer.decode(outputs[0],skip_special_tokens=True))

<IPython.core.display.Latex object>

___

# RAG with refined text

In [4]:
loader = TextLoader("text.txt", encoding="utf-8")
docs=loader.load()

In [5]:
splitter=RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

/tmp/ipykernel_2199991/3384329387.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
ques="""A square-thread power screw has a major diameter of 64 mm and a pitch of 4 mm with double threads, and it is to be used in screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 8 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load."""

similar=vectorstore.similarity_search(ques)
context=""
for txt in similar:
    context+=txt.page_content+"\n"

content=f"""Using the following context:<context>{context}<\context>\n
Answer the following question:<question>{ques}<\question>\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text=tokenizer.decode(input_tensor[0], skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=3000)

In [7]:
Latex(tokenizer.decode(outputs[0],skip_special_tokens=True))

<IPython.core.display.Latex object>

In [8]:
splitter=TokenTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

In [10]:
ques="""A square-thread power screw has a major diameter of 64 mm and a pitch of 4 mm with double threads, and it is to be used in screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 8 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load."""

similar=vectorstore.similarity_search(ques)
context=""
for txt in similar:
    context+=txt.page_content+"\n"

content=f"""Usning thefollowing context:<context>{context}<\context>\n
Answer the following question:<question>{ques}<\question>\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text=tokenizer.decode(input_tensor[0], skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=3000)

In [11]:
Latex(tokenizer.decode(outputs[0], skip_special_tokens=True))

<IPython.core.display.Latex object>

In [12]:
splitter=SpacyTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

/home/vikram/Downloads/aban/.venv/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
ques="""A square-thread power screw has a major diameter of 64 mm and a pitch of 4 mm with double threads, and it is to be used in screw jack.
The given data include f = fc = 0.08, dc = 40 mm, and F = 8 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load."""

similar=vectorstore.similarity_search(ques)
context=""
for txt in similar:
    context+=txt.page_content+"\n"

content=f"""Using the following context:<context>{context}<\context>\n
Answer the following question:<question>{ques}<\question>\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input_text=tokenizer.decode(input_tensor[0], skip_special_tokens=True)
input=tokenizer(input_text,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=3000)

In [14]:
Latex(tokenizer.decode(outputs[0], skip_special_tokens=True))

<IPython.core.display.Latex object>

---

In [7]:
context=r"""the torque required to turn the screw against the load is\n
$T_R=\frac{F.d_m}{2}(\frac{l + \pi.f.d_m}{\pi.d_m - f.l})+\frac{F.f_c.d_c}{2}$\n
the load-lowering torque is\n
$T_L=\frac{F.d_m}{2}(\frac{\pi.f.d_m - l}{\pi.d_m + f.l})+\frac{F.f_c.d_c}{2}$\n

where: l=lead (is the distance the nut moves parallel to the screw axis when the nut is given one turn.) =n*p
$d_m$=pitch diameter in mm = major diameter- pitch/2

thread depth=thread width, for a square thread screw. 
$efficiency=\frac{F*l}{2*\pi*T_R}$
"""

In [ ]:
input_text = f"""Using the following context:<context>{context}<\context>
Solve the following question:<question>
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load.<\question>\n
First reason then give your answer in steps. Do the calculation in steps."""
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = deepseek_model.generate(**input_ids,max_new_tokens=2000)
# print(tokenizer.decode(outputs[0]))
Latex(tokenizer.decode(outputs[0],skip_special_tokens=True))

<IPython.core.display.Latex object>

In [8]:
input_text = f"""Using the following context:<context>{context}<\context>
Solve the following question:<question>
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load.
(c) Find the efficiency during lifting the load.<\question>
Do step by step calculation."""
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = deepseek_model.generate(**input_ids,max_new_tokens=2000)
# print(tokenizer.decode(outputs[0]))
Latex(tokenizer.decode(outputs[0],skip_special_tokens=True))

<IPython.core.display.Latex object>

---

# Graph text

In [ ]:
loader = TextLoader("graph_text.txt", encoding="utf-8")
docs=loader.load()

In [ ]:
splitter=SpacyTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

In [ ]:
ques="""A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load.
(c) Find the efficiency during lifting the load."""

similar=vectorstore.similarity_search(ques)
context=""
for txt in similar:
    context+=txt.page_content+"\n"

content=f"""Using the following context:<context>{context}<\context>\n
Answer the following question:<question>{ques}<\question>\n
Give your answer with step by step reasoning"""
messages = [
    {"role": "user", "content": content}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
input=tokenizer(input_tensor,return_tensors='pt').to(deepseek_model.device)
outputs = deepseek_model.generate(**input, max_new_tokens=3000)
Latex(tokenizer.decode(outputs[0]))

---

### with calculator tool

In [ ]:
@tool
def calculator(expression: str) -> str:
    """Evaluates a basic mathematical expression like 2+3*4"""
    try:
        return str(eval(expression, {"__builtins__": {}}, {}))
    except Exception as e:
        print(e)
        return f"Error: {e}"

tools = [Tool.from_function(calculator, calculator.name, calculator.description)]

hf_pipe = pipeline("text-generation", model=deepseek_model, tokenizer=tokenizer, max_new_tokens=2000)
llm = HuggingFacePipeline(pipeline=hf_pipe)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a expert mechanical engineer advisor. Use the calculator tool when needed."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])
tools=[calculator]
agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent="conversational-react-description",
    verbose=True
)

In [ ]:
input_text = f"""Using the following context:<context>{context}<\context>
Solve the following question:<question>
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load.
(c) Find the efficiency during lifting the load.<\question>
Do step by step calculation."""
res=agent.invoke({"input":input_text,"agent_scratchpad":[],"chat_history":[]})

### with meta data

In [ ]:
loader = TextLoader("meta_data.txt", encoding="utf-8")
docs=loader.load()

splitter=RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

retriever=vectorstore.as_retriever()

hf_pipe = pipeline("text-generation", model=deepseek_model, tokenizer=tokenizer, max_new_tokens=2000)
llm = HuggingFacePipeline(pipeline=hf_pipe)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    retriever=retriever,
)

input_text = f"""Solve the following question:<question>
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load.
(c) Find the efficiency during lifting the load.<\question>
Do step by step calculation."""
res=agent.invoke({"input":input_text,"agent_scratchpad":[],"chat_history":[]})


In [ ]:
loader = TextLoader("meta_data.txt", encoding="utf-8")
docs=loader.load()

splitter=RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
chunks=splitter.split_documents(docs)

embedding_model=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

retriever=vectorstore.as_retriever()

hf_pipe = pipeline("text-generation", model=deepseek_model, tokenizer=tokenizer, max_new_tokens=2000)
llm = HuggingFacePipeline(pipeline=hf_pipe)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    retriever=retriever,
)

input_text = f"""Solve the following question:<question>
A square-thread power screw has a major diameter of 32 mm and a pitch of 4 mm with double threads.
The given data include f = fc = 0.08, dc = 40 mm, and F = 6.4 kN per screw.
(a) Find the thread depth, thread width, pitch diameter, minor diameter, and lead.
(b) Find the torque required to raise and lower the load.
(c) Find the efficiency during lifting the load.
(d) Find the body stresses, torsional and compressive.
(e) Find the bearing stress.
(f) Find the thread bending stress at the root of the thread.
(g) Determine the von Mises stress at the root of the thread.
(h) Determine the maximum shear stress at the root of the thread.<\question>
Do step by step calculation."""
res=agent.invoke({"input":input_text,"agent_scratchpad":[],"chat_history":[]})